# Remote DCOM IErtUtil DLL Hijack

## Metadata


|     Metadata      |  Value  |
|:------------------|:---|
| collaborators     | ['@Cyb3rWard0g', '@Cyb3rPandaH'] |
| creation date     | 2020/10/09 |
| modification date | 2020/10/09 |
| playbook related  | ['WIN-201012004336'] |

## Hypothesis
Threat actors might be copying files remotely to abuse a DLL hijack opportunity found on the DCOM InternetExplorer.Application Class.

## Technical Context


## Offensive Tradecraft
A threat actor could use a known DLL hijack vulnerability on the DCOM InternetExplorer.Application Class while instantiating the object remotely.
When the object instantiate, it looks for `iertutil.dll` in the `c:\Program Files\Internet Explorer\` directory. That DLL does not exist in that folder. Therefore, a threat actor could easily copy its own DLL in that folder and execute it by instantiating an object via the DCOM InternetExplorer.Application Class remotely.
When the malicious DLL is loaded, there are various approaches to hijacking execution, but most likely a threat actor would want the DLL to act as a proxy to the real DLL to minimize the chances of interrupting normal operations.
One way to do this is by cloning the export table from one DLL to another one. One known tool that can help with it is [Koppeling](https://github.com/monoxgas/Koppeling). 

## Security Datasets


| Metadata  |    Value  |
|:----------|:----------|
| docs      | https://securitydatasets.com/notebooks/atomic/windows/lateral_movement/SDWIN-201009183000.html        |
| link      | [https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/lateral_movement/host/covenant_dcom_iertutil_dll_hijack.zip](https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/lateral_movement/host/covenant_dcom_iertutil_dll_hijack.zip)  |

## Analytics

### Initialize Analytics Engine

In [ ]:
from openhunt.mordorutils import *
spark = get_spark()

### Download & Process Security Dataset

In [ ]:
sd_file = "https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/lateral_movement/host/covenant_dcom_iertutil_dll_hijack.zip"
registerMordorSQLTable(spark, sd_file, "sdTable")

### Analytic I
Look for non-system accounts SMB accessing a `C:\Program Files\Internet Explorer\iertutil.dll` with write (0x2) access mask via an administrative share (i.e C$).


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| File | Microsoft-Windows-Security-Auditing | User accessed File | 5145 |

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, Hostname, ShareName, SubjectUserName, SubjectLogonId, IpAddress, IpPort, RelativeTargetName
FROM sdTable
WHERE LOWER(Channel) = "security"
    AND EventID = 5145
    AND RelativeTargetName LIKE '%Internet Explorer\\\iertutil.dll'
    AND NOT SubjectUserName LIKE '%$'
    AND AccessMask = '0x2'
'''
)
df.show(10,False)

### Analytic II
Look for `C:\Program Files\Internet Explorer\iertutil.dll` being accessed over the network with write (0x2) access mask via an administrative share (i.e C$) and created by the System process on the target system.


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| File | Microsoft-Windows-Security-Auditing | User accessed File | 5145 |
| File | Microsoft-Windows-Sysmon/Operational | Process created File | 11 |

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, Hostname, ShareName, SubjectUserName, SubjectLogonId, IpAddress, IpPort, RelativeTargetName
FROM sdTable b
INNER JOIN (
    SELECT LOWER(REVERSE(SPLIT(TargetFilename, '\'))[0]) as TargetFilename
    FROM sdTable
    WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
        AND Image = 'System'
        AND EventID = 11
        AND TargetFilename LIKE '%Internet Explorer\\\iertutil.dll'
) a
ON LOWER(REVERSE(SPLIT(RelativeTargetName, '\'))[0]) = a.TargetFilename
WHERE LOWER(b.Channel) = 'security'
    AND b.EventID = 5145
    AND b.AccessMask = '0x2'
'''
)
df.show(10,False)

### Analytic III
Look for `C:\Program Files\Internet Explorer\iertutil.dll` being accessed over the network with write (0x2) access mask via an administrative share (i.e C$) and created by the System process on the target system.


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| File | Microsoft-Windows-Security-Auditing | User accessed File | 5145 |
| File | Microsoft-Windows-Sysmon/Operational | Process created File | 11 |

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, Hostname, ShareName, SubjectUserName, SubjectLogonId, IpAddress, IpPort, RelativeTargetName
FROM sdTable b
INNER JOIN (
    SELECT LOWER(REVERSE(SPLIT(TargetFilename, '\'))[0]) as TargetFilename
    FROM sdTable
    WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
        AND Image = 'System'
        AND EventID = 11
        AND TargetFilename LIKE '%Internet Explorer\\\iertutil.dll'
) a
ON LOWER(REVERSE(SPLIT(RelativeTargetName, '\'))[0]) = a.TargetFilename
WHERE LOWER(b.Channel) = 'security'
    AND b.EventID = 5145
    AND b.AccessMask = '0x2'
'''
)
df.show(10,False)

### Analytic IV
Look for `C:\Program Files\Internet Explorer\iertutil.dll` being accessed over the network with write (0x2) access mask via an administrative share (i.e C$), created by the System process and loaded by the WMI provider host (wmiprvse.exe). All happening on the target system.


| Data source | Event Provider | Relationship | Event |
|:------------|:---------------|--------------|-------|
| File | Microsoft-Windows-Security-Auditing | User accessed File | 5145 |
| File | Microsoft-Windows-Sysmon/Operational | Process created File | 11 |
| File | Microsoft-Windows-Sysmon/Operational | Process loaded Dll | 7 |

In [ ]:
df = spark.sql(
'''
SELECT `@timestamp`, Hostname, ShareName, SubjectUserName, SubjectLogonId, IpAddress, IpPort, RelativeTargetName
FROM sdTable d
INNER JOIN (
    SELECT LOWER(REVERSE(SPLIT(TargetFilename, '\'))[0]) as TargetFilename
    FROM sdTable b
    INNER JOIN (
        SELECT ImageLoaded
        FROM sdTable
        WHERE Channel = 'Microsoft-Windows-Sysmon/Operational'
            AND EventID = 7
            AND LOWER(Image) LIKE '%iexplore.exe'
            AND ImageLoaded LIKE '%Internet Explorer\\\iertutil.dll'
    ) a
    ON b.TargetFilename = a.ImageLoaded
    WHERE b.Channel = 'Microsoft-Windows-Sysmon/Operational'
        AND b.Image = 'System'
        AND b.EventID = 11
) c
ON LOWER(REVERSE(SPLIT(RelativeTargetName, '\'))[0]) = c.TargetFilename
WHERE LOWER(d.Channel) = 'security'
    AND d.EventID = 5145
    AND d.AccessMask = '0x2'
'''
)
df.show(10,False)

## Known Bypasses

## False Positives
None

## Hunter Notes
* Baseline your environment to identify normal activity. Document all accounts creating files over the network via administrative shares.
* Baseline iexplore.exe execution and modules loaded (i.e signed and un-signed)

## Hunt Output

| Type | Link |
| :----| :----|
| Sigma Rule | [https://github.com/OTRF/ThreatHunter-Playbook/blob/master/signatures/sigma/win_dcom_iertutil_dll_hijack.yml](https://github.com/OTRF/ThreatHunter-Playbook/blob/master/signatures/sigma/win_dcom_iertutil_dll_hijack.yml) |
| Sigma Rule | [https://github.com/OTRF/ThreatHunter-Playbook/blob/master/signatures/sigma/sysmon_dcom_iertutil_dll_hijack.yml](https://github.com/OTRF/ThreatHunter-Playbook/blob/master/signatures/sigma/sysmon_dcom_iertutil_dll_hijack.yml) |

## References
* https://docs.microsoft.com/en-us/openspecs/windows_protocols/ms-dcom/64af4c57-5466-4fdf-9761-753ea926a494